# 判别器（Discriminator）详解与演示

本教程将详细讲解 PyTorch 中判别器的实现和使用。判别器是生成对抗网络（GAN）中的重要组成部分，其主要作用是区分真实数据和生成的数据。

在本例中，我们将实现一个能够对高维特征向量进行分类的判别器，这些向量可以是句子或其他类型的数据的表示。

## 1. 导入必要的库

首先，我们需要导入 PyTorch 相关的库和可视化工具。

In [2]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 设置随机种子以保证结果可复现
torch.manual_seed(42)

# 设置显示中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']  # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号

## 2. 定义判别器类

下面我们将定义判别器类。这个判别器包含以下关键组件：
- 全连接层（Linear layers）
- LeakyReLU 激活函数
- Dropout 层用于防止过拟合
- Softmax 层用于输出概率分布

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, 
                 input_size,          # 输入特征维度（如 BERT 输出的 [CLS] 向量）
                 hidden_sizes,      # 隐藏层结构
                 num_labels,            # 实际类别数（不包含
                 dropout_rate):       # dropout 防止过拟合
        super(Discriminator, self).__init__()

        # 输入特征的 dropout 层
        self.input_dropout = nn.Dropout(p=dropout_rate)

        # 构建隐藏层网络结构
        layers = []
        hidden_sizes = [input_size] + hidden_sizes  # 构造层次，如 [768, 768]
        for i in range(len(hidden_sizes) - 1):
            layers.extend([
                nn.Linear(hidden_sizes[i], hidden_sizes[i+1]),     # 全连接层
                nn.LeakyReLU(0.2, inplace=True),                   # 激活函数（相比 ReLU 更稳定）
                nn.Dropout(dropout_rate)                           # Dropout
            ])

        # 封装为顺序模型
        self.layers = nn.Sequential(*layers)

        # 最后输出层：k 个真实类别 + 1 个
        self.logit = nn.Linear(hidden_sizes[-1], num_labels + 1)

        # softmax 函数把原始模型输出（通常是未经缩放的“logits”）转换为各类别的概率；
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, input_rep):
        """
        输入：input_rep 是来自 BERT 或生成器的句子向量表示
        输出：
          - last_rep: 判别器提取的深层特征（用于 feature matching）
          - logits: 未归一化的分类分数
          - probs: softmax 后的预测概率（包括第 k+1 类）
        """
        input_rep = self.input_dropout(input_rep)
        last_rep = self.layers(input_rep)       # 中间特征表示
        logits = self.logit(last_rep)           # 分类器输出 logits
        probs = self.softmax(logits)            # softmax 得到概率分布  

        return last_rep, logits, probs

In [13]:
# 创建判别器实例
discriminator = Discriminator(
    input_size=768,     # 输入维度为768（与BERT等预训练模型的隐藏层维度相匹配）
    hidden_sizes=[768,1028,768], # 三层隐藏层
    num_labels=5,       # 分类数为5
    dropout_rate=0.1    # 10%的dropout率
)

# 打印判别器结构以查看网络架构
print("判别器结构：")
print(discriminator)

判别器结构：
Discriminator(
  (input_dropout): Dropout(p=0.1, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=768, out_features=768, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=768, out_features=1028, bias=True)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=1028, out_features=768, bias=True)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Dropout(p=0.1, inplace=False)
  )
  (logit): Linear(in_features=768, out_features=6, bias=True)
  (softmax): Softmax(dim=-1)
)


## 4. 测试判别器的功能

现在我们将创建一些模拟的输入数据，并通过判别器进行处理，观察输出结果。

In [14]:
# 创建一个batch的模拟输入数据（假设batch_size=16）
batch_size = 16
mock_input = torch.randn(batch_size, 768)  # 随机生成16个768维的向量

# 通过判别器处理数据
with torch.no_grad():  # 不计算梯度
    last_rep, logits, probs = discriminator(mock_input)

# 打印结果
print("\n中间特征维度：", last_rep.shape)
print("\n分类logits维度：\n", logits.shape)
print("\n预测概率维度：\n", probs.shape)




中间特征维度： torch.Size([16, 768])

分类logits维度：
 torch.Size([16, 6])

预测概率维度：
 torch.Size([16, 6])
